In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import zipfile
import matplotlib.dates as mdates


def unzip_and_rename_csv(zip_file, extract_folder=None, new_name="latest_od_data.csv"):
    """
    Extracts a CSV from a ZIP file, renames it, and returns its path.
    """
    extract_folder = extract_folder or os.path.dirname(zip_file)
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
        csv_file = next((os.path.join(extract_folder, f) for f in zip_ref.namelist() if f.lower().endswith('.csv')), None)
    if not csv_file:
        raise FileNotFoundError("No CSV found in the ZIP.")
    renamed_path = os.path.join(extract_folder, new_name)
    os.rename(csv_file, renamed_path)
    return renamed_path


# Load the most recent CSV or extract from ZIP
csv_files = glob.glob(os.path.expanduser('~/Downloads/*.csv'))
latest_csv = max(csv_files, key=os.path.getmtime) if csv_files else unzip_and_rename_csv('./od_logs/latest_od_data.zip')
exp1_csv = unzip_and_rename_csv('./od_logs/exp1.zip', new_name="exp1.csv")

# Load and preprocess data
def preprocess_csv(file_path):
    """Load and preprocess CSV data."""
    return (
        pd.read_csv(file_path)
        .rename(columns={"od_reading": "OD600", "timestamp_localtime": "Time", "pioreactor_unit": "Unit"})
        .drop(columns=["experiment", "timestamp", "angle", "channel"])
        .assign(Time=lambda d: pd.to_datetime(d["Time"]))
    )

df = pd.concat([preprocess_csv(latest_csv), preprocess_csv(exp1_csv)])
df = df.drop_duplicates(subset=['Time', 'Unit'])  # Remove duplicates

# Pivot for analysis
df = df.pivot(index='Time', columns='Unit', values='OD600')

# Remove the 'Unit' name from columns
df.columns.name = None

# Apply transformations
transformations = {
    'worker2': (1.3486, 0.077),
    'worker3': (2.0737, 0.077),
    'worker4': (2.2043, 0.077),
    'worker5': (1.1695, 0.077)
}

for unit, (slope, intercept) in transformations.items():
    if unit in df.columns:
        df[unit] = df[unit] * slope + intercept

# Filter out datapoints above 0.45
df = df[df < 0.45]
df = df[100:]

df = df.rename(columns={"worker2": "w2 - 35 cntrl", "worker3" : "w3 - rep2", "worker4": "w4 - rep1", "worker5" : "w5 - 55 cntrl"})

for worker in df:
    if worker not in df.columns:
        continue  # Skip if the worker column does not exist

    # Resample and calculate the rolling standard deviation
    data = df[[worker]].resample('30min').mean()

    # Identify subculture points
    subculture_points = data.loc[data[worker].rolling(3).std().abs() > 0.02]
    subculture_points = subculture_points.groupby(subculture_points.index.date).idxmin().reset_index(drop=True).to_numpy().flatten()

    # Resample for visualization
    data = data.resample('10min').mean()

    # Create new subculture windows
    new_data = pd.DataFrame()
    start_index = data.index[0]

    for i, point in enumerate(subculture_points):
        current_step = data.loc[start_index:point]
        current_step = current_step.iloc[:-6]
        current_step.index = ((current_step.index - current_step.index[0]).total_seconds() / 60).astype(int)
        current_step.columns = [f'{worker}_S{i+1}']
        new_data = pd.concat([new_data, current_step], axis=1)
        if i == len(subculture_points) - 1:
            break
        start_index = point

    new_data.index.name = 'Min'
    new_data = new_data.interpolate(method='linear', limit_area='inside').round(4)
    new_data = new_data.fillna(0)

    display(new_data)

    # Save the new data to a file if needed
    # new_data.to_csv(f'{worker}.txt', sep='\t')

,w2 - 35 cntrl_S1,w2 - 35 cntrl_S2,w2 - 35 cntrl_S3,w2 - 35 cntrl_S4,w2 - 35 cntrl_S5,w2 - 35 cntrl_S6,w2 - 35 cntrl_S7,w2 - 35 cntrl_S8,w2 - 35 cntrl_S9,w2 - 35 cntrl_S10,w2 - 35 cntrl_S11,w2 - 35 cntrl_S12,w2 - 35 cntrl_S13,w2 - 35 cntrl_S14,w2 - 35 cntrl_S15,w2 - 35 cntrl_S16,w2 - 35 cntrl_S17
Min,,,,,,,,,,,,,,,,,
0,0.0827,0.0842,0.0821,0.0809,0.0813,0.0830,0.0816,0.0816,0.0826,0.0796,0.0804,0.0807,0.0806,0.0803,0.0809,0.0816,0.0826
10,0.0826,0.0843,0.0822,0.0809,0.0813,0.0831,0.0816,0.0817,0.0824,0.0796,0.0801,0.0807,0.0807,0.0803,0.0809,0.0816,0.0826
20,0.0825,0.0843,0.0823,0.0810,0.0813,0.0831,0.0815,0.0819,0.0821,0.0796,0.0798,0.0808,0.0808,0.0803,0.0810,0.0817,0.0826
30,0.0824,0.0844,0.0824,0.0810,0.0812,0.0831,0.0815,0.0820,0.0818,0.0797,0.0795,0.0808,0.0809,0.0802,0.0811,0.0817,0.0826
40,0.0823,0.0845,0.0825,0.0811,0.0813,0.0831,0.0815,0.0820,0.0818,0.0797,0.0796,0.0809,0.0808,0.0801,0.0810,0.0817,0.0827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2900,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.3314
2910,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.3308
2920,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.2906


,w3 - rep2_S1,w3 - rep2_S2,w3 - rep2_S3,w3 - rep2_S4,w3 - rep2_S5,w3 - rep2_S6,w3 - rep2_S7,w3 - rep2_S8,w3 - rep2_S9,w3 - rep2_S10
Min,,,,,,,,,,
0,0.0806,0.0818,0.0807,0.0820,0.0803,0.0818,0.0791,0.0809,0.0797,0.0814
10,0.0806,0.0819,0.0808,0.0820,0.0804,0.0818,0.0791,0.0808,0.0798,0.0815
20,0.0806,0.0819,0.0809,0.0820,0.0805,0.0819,0.0792,0.0807,0.0799,0.0815
30,0.0806,0.0820,0.0810,0.0819,0.0805,0.0820,0.0792,0.0807,0.0800,0.0815
40,0.0806,0.0820,0.0810,0.0819,0.0804,0.0819,0.0792,0.0805,0.0799,0.0815
...,...,...,...,...,...,...,...,...,...,...
7760,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.1843,0.0000,0.0000
7770,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.1841,0.0000,0.0000
7780,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.1842,0.0000,0.0000


,w4 - rep1_S1,w4 - rep1_S2,w4 - rep1_S3,w4 - rep1_S4,w4 - rep1_S5,w4 - rep1_S6,w4 - rep1_S7,w4 - rep1_S8,w4 - rep1_S9,w4 - rep1_S10,w4 - rep1_S11
Min,,,,,,,,,,,
0,0.0836,0.0825,0.0816,0.0834,0.0812,0.0818,0.0818,0.0830,0.0826,0.0827,0.0816
10,0.0834,0.0825,0.0818,0.0833,0.0813,0.0822,0.0819,0.0831,0.0826,0.0829,0.0819
20,0.0832,0.0826,0.0819,0.0833,0.0814,0.0826,0.0819,0.0832,0.0826,0.0831,0.0821
30,0.0830,0.0826,0.0820,0.0832,0.0816,0.0830,0.0819,0.0833,0.0825,0.0832,0.0824
40,0.0831,0.0825,0.0822,0.0832,0.0815,0.0828,0.0821,0.0832,0.0827,0.0832,0.0826
...,...,...,...,...,...,...,...,...,...,...,...
7160,0.0000,0.0000,0.0000,0.0000,0.0000,0.2101,0.0000,0.0000,0.0000,0.0000,0.0000
7170,0.0000,0.0000,0.0000,0.0000,0.0000,0.2098,0.0000,0.0000,0.0000,0.0000,0.0000
7180,0.0000,0.0000,0.0000,0.0000,0.0000,0.2098,0.0000,0.0000,0.0000,0.0000,0.0000


,w5 - 55 cntrl_S1,w5 - 55 cntrl_S2,w5 - 55 cntrl_S3,w5 - 55 cntrl_S4,w5 - 55 cntrl_S5,w5 - 55 cntrl_S6,w5 - 55 cntrl_S7,w5 - 55 cntrl_S8,w5 - 55 cntrl_S9
Min,,,,,,,,,
0,0.0806,0.0806,0.0788,0.0806,0.0806,0.0795,0.0801,0.0809,0.0798
10,0.0808,0.0807,0.0788,0.0807,0.0808,0.0796,0.0802,0.0810,0.0798
20,0.0810,0.0808,0.0788,0.0807,0.0810,0.0797,0.0802,0.0810,0.0799
30,0.0811,0.0808,0.0788,0.0808,0.0812,0.0797,0.0803,0.0811,0.0799
40,0.0810,0.0808,0.0788,0.0808,0.0811,0.0797,0.0803,0.0811,0.0800
...,...,...,...,...,...,...,...,...,...
5780,0.0000,0.0000,0.0000,0.0000,0.2161,0.0000,0.0000,0.0000,0.0000
5790,0.0000,0.0000,0.0000,0.0000,0.2160,0.0000,0.0000,0.0000,0.0000
5800,0.0000,0.0000,0.0000,0.0000,0.2161,0.0000,0.0000,0.0000,0.0000
